In [ ]:
import gymnasium as gym
from simpler_env.utils.env.observation_utils import get_image_from_maniskill3_obs_dict
from ManiSkill.mani_skill.envs.tasks.digital_twins.bridge_dataset_eval import *
env = gym.make(
  "PutSpoonOnTableClothInScene-v1",
  obs_mode="rgb+segmentation",
  num_envs=16, # if num_envs > 1, GPU simulation backend is used.
)
obs, _ = env.reset()
# returns language instruction for each parallel env
instruction = env.unwrapped.get_language_instruction()
print("instruction:", instruction[0])

while True:
  # action[:3]: delta xyz; action[3:6]: delta rotation in axis-angle representation;
  # action[6:7]: gripper (the meaning of open / close depends on robot URDF)
  image = get_image_from_maniskill3_obs_dict(env, obs) # this is the image observation for policy inference
  action = env.action_space.sample() # replace this with your policy inference
  obs, reward, terminated, truncated, info = env.step(action)
  if truncated.any():
      break
print("Episode Info", info)

In [ ]:
import simpler_env
from simpler_env.utils.env.observation_utils import get_image_from_maniskill3_obs_dict
import mediapy
import sapien.core as sapien

task_name = "google_robot_pick_coke_can"  # @param ["google_robot_pick_coke_can", "google_robot_move_near", "google_robot_open_drawer", "google_robot_close_drawer", "widowx_spoon_on_towel", "widowx_carrot_on_plate", "widowx_stack_cube", "widowx_put_eggplant_in_basket"]

if 'env' in locals():
  print("Closing existing env")
  env.close()
  del env
env = simpler_env.make(task_name)
env = env.env.env.env
# Colab GPU does not supoort denoiser
sapien.render_config.rt_use_denoiser = False
obs, reset_info = env.reset()
instruction = env.get_language_instruction()
print("Reset info", reset_info)
print("Instruction", instruction)

frames = []
done, truncated = False, False
idx = 0
while not (done or truncated):
    # action[:3]: delta xyz; action[3:6]: delta rotation in axis-angle representation;
    # action[6:7]: gripper (the meaning of open / close depends on robot URDF)
    image = get_image_from_maniskill3_obs_dict(env, obs)
    action = env.action_space.sample() # replace this with your policy inference
    print(action)
    obs, reward, done, truncated, info = env.step(action)
    print("===============")
    print("Reward", reward)
    print("obs", obs)
    print("done", done)
    print("truncated", truncated)
    print("info", info)
    print("===============")
    frames.append(image)
    idx += 1
    # print(idx, end=' ')
    if idx == 2:
        break

episode_stats = info.get('episode_stats', {})
print("Episode stats", episode_stats)
import numpy as np
np_frames = np.array(frames)
frames = np_frames[..., :3]
mediapy.show_video(
    frames,
    fps=10,
    # codec='gif'
)